In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Carregando os dados
df = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/train.csv')
test = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/test.csv')

df.shape, test.shape

In [ ]:
# Juntando os dataframes de treino e teste
df_all = df.append(test)

df_all.shape

In [ ]:
# Entender os dados
df_all.info(max_cols=145)

## Pré-processamento e limpeza dos dados

In [ ]:
# Quais colunas do dataframe são do tipo object
df_all.select_dtypes('object').head()

In [ ]:
# Olhando a coluna dependency
df_all['dependency'].value_counts()

##### coluna dependency, Dependency rate, calculated = (number of members of the household younger than 19 or older than 64)/(number of member of household between 19 and 64)

### Investigando o valor 'yes' na coluna dependency

In [ ]:
# Vamos obter o idhogar de alguns casos
df_all[df_all['dependency'] == 'yes']['idhogar'].head()

In [ ]:
# Vamos investigar o idhogar 2b58d945f pegando as variáveis de cálculo da coluna dependency
df_all[df_all['idhogar'] == '2b58d945f'][['hogar_nin', 'hogar_adul', 'hogar_mayor', 'hogar_total']]

In [ ]:
# Vamos investigar o idhogar d6dae86b7 pegando as variáveis de cálculo da coluna dependency
df_all[df_all['idhogar'] == 'd6dae86b7'][['hogar_nin', 'hogar_adul', 'hogar_mayor', 'hogar_total']]

### dependency Yes está dando 1
#### avaliar o no agora

In [ ]:
# Vamos obter o idhogar de alguns casos
df_all[df_all['dependency'] == 'no']['idhogar'].head()

In [ ]:
# Vamos investigar o idhogar 3e16fab89 pegando as variáveis de cálculo da coluna dependency
df_all[df_all['idhogar'] == '3e16fab89'][['hogar_nin', 'hogar_adul', 'hogar_mayor', 'hogar_total']]

In [ ]:
# Vamos investigar o idhogar d29058053 pegando as variáveis de cálculo da coluna dependency
df_all[df_all['idhogar'] == 'd29058053'][['hogar_nin', 'hogar_adul', 'hogar_mayor', 'hogar_total']]

### dependency 'no' está dando 0

* edjefe, years of education of male head of household, based on the interaction of escolari (years of education), head of household and gender, yes=1 and no=0
* edjefa, years of education of female head of household, based on the interaction of escolari (years of education), head of household and gender, yes=1 and no=0

In [ ]:
# Analisando os dados da coluna edjefa
df_all['edjefa'].value_counts()

In [ ]:
# Analisando os dados da coluna edjefe
df_all['edjefe'].value_counts()

In [ ]:
# Vamos transformar 'yes' em 1 e 'no' em 0
# nas colunas edjefa e edjefe
mapeamento = {'yes': 1, 'no': 0}

df_all['edjefa'] = df_all['edjefa'].replace(mapeamento).astype(int)
df_all['edjefe'] = df_all['edjefe'].replace(mapeamento).astype(int)

In [ ]:
# Vamos transformar 'yes' em 1 e 'no' em 0
# na coluna dependency
df_all['dependency'] = df_all['dependency'].replace(mapeamento).astype(float)

In [ ]:
# Quais colunas do dataframe são do tipo object
df_all.select_dtypes('object').head()

In [ ]:
# Visualizando do comando info
df_all.info(max_cols=145)

In [ ]:
# Verificando os valores nulos
df_all.isnull().sum()

### avaliando Colunas com NA

* v2a1, Monthly rent payment
* v18q, owns a tablet
* v18q1, number of tablets household owns
* rez_esc, Years behind in school
* meaneduc,average years of education for adults (18+)
* SQBmeaned, square of the mean years of education of adults (>=18) in the household

In [ ]:
# Verificando os valores de aluguel (v2a1) para os chefes/as de familia (parentesco1 = 1)
df_all[df_all['parentesco1'] == 1]['v2a1'].isnull().sum()

In [ ]:
# Verificando os valores de aluguel (v2a1) para 
df_all[df_all['parentesco2'] == 1]['v2a1'].isnull().sum()

In [ ]:
# Verificando os valores de aluguel (v2a1) para 
df_all[df_all['parentesco3'] == 1]['v2a1'].isnull().sum()

In [ ]:
# Verificando os valores de aluguel (v2a1) para 
df_all[df_all['parentesco12'] == 1]['v2a1'].isnull().sum()

In [ ]:
# Qual a cara dos dados de v18q
df_all['v18q'].value_counts()

### v18q1 tem 7945 valores, ou seja, as colunas com NA é quando o valor de tablet é 0

In [ ]:
# Prenchendo com 0 os valores nulos de v18q1
df_all['v18q1'].fillna(0, inplace=True)

In [ ]:
# Qual a cara dos dados de escolari
df_all['escolari'].value_counts()

In [ ]:
# Qual a cara dos dados de rez_esc
df_all['rez_esc'].value_counts()

In [ ]:

df_all['meaneduc'].value_counts()

In [ ]:
df_all[df_all['meaneduc'].isnull()][['hogar_nin', 'hogar_adul', 'hogar_mayor', 'hogar_total']]

In [ ]:
df_all[df_all['meaneduc'].isnull()][['escolari', 'SQBmeaned']]

In [ ]:
df_all.loc[(df_all['meaneduc'].isnull()) & (df_all['escolari'] == 0)][['meaneduc', 'escolari']]

In [ ]:
mask = (df_all['meaneduc'].isnull()) & (df_all['escolari'] == 0)

df_all.loc[mask, 'meaneduc'] = 0
df_all.loc[mask, 'SQBmeaned'] = 0

In [ ]:
df_all[df_all['meaneduc'].isnull()][['hogar_nin', 'hogar_adul', 'hogar_mayor', 'hogar_total']]

In [ ]:
df_all[df_all['meaneduc'].isnull()][['escolari', 'SQBmeaned']]

* as colunas que 'escolari' era zero, substitui o NA do 'meaneduc' e do 'SQBmeaned' para 0

In [ ]:
df_all[['escolari', 'meaneduc', 'hogar_nin', 'hogar_adul', 'hogar_mayor', 'hogar_total', 'age', 'idhogar']]

In [ ]:
df_all[df_all['idhogar'] == '3aa78c56b'][['hogar_nin', 'hogar_adul', 'hogar_mayor', 'hogar_total','age', 'escolari', 'idhogar', 'meaneduc']]

'meaneduc' é a soma da escolaridade dos adultos > 19 anos / pelo numero de adultos
(5+5+5+8)/4 =5.75

In [ ]:
df_all[df_all['meaneduc'].isnull()][['hogar_nin', 'hogar_adul', 'hogar_mayor', 'hogar_total','age', 'escolari', 'meaneduc',  'SQBmeaned']]

* como 'hogar_adul' é 0 em todos os casos que o 'meaneduc' é NA, 'meaneduc' é 0 nessas linhas e consequentemente 'SQBmeaned' é 0 nessas linhas
* o dicionário de dados está errado ao falar que 'SQBmeaned' é de adultos >= 18, na verdade adulto pra base é > 18

In [ ]:
df_all['SQBmeaned'].fillna(0, inplace=True)
df_all['meaneduc'].fillna(0, inplace=True)

In [ ]:
df_all.info(max_cols=145)

In [ ]:
# vendo a dis†ribuição de lugares em que o valor do aluguel 'v2a1' é NA

df_all[df_all['v2a1'].isnull()][['lugar1', 'lugar2', 'lugar3', 'lugar4', 'lugar5', 'lugar6']].value_counts()

In [ ]:
# vendo a dis†ribuição de lugares em que o valor do aluguel 'v2a1' não é NA

df_all[df_all['v2a1'].notnull()][['lugar1', 'lugar2', 'lugar3', 'lugar4', 'lugar5', 'lugar6']].value_counts()

In [ ]:
# vendo a dis†ribuição de lugares em que o valor do aluguel 'v2a1' não é NA
df_s =df_all[df_all['v2a1'].notnull()]

df_s.groupby(['lugar1', 'lugar2', 'lugar3', 'lugar4', 'lugar5', 'lugar6'])[['v2a1']].median()

In [ ]:
mask_lugar1 = (df_all['v2a1'].isnull()) & (df_all['lugar1'] == 1)
mask_lugar2 = (df_all['v2a1'].isnull()) & (df_all['lugar2'] == 1)
mask_lugar3 = (df_all['v2a1'].isnull()) & (df_all['lugar3'] == 1)
mask_lugar4 = (df_all['v2a1'].isnull()) & (df_all['lugar4'] == 1)
mask_lugar5 = (df_all['v2a1'].isnull()) & (df_all['lugar5'] == 1)
mask_lugar6 = (df_all['v2a1'].isnull()) & (df_all['lugar6'] == 1)

df_all.loc[mask_lugar1, 'v2a1'] = 155000.0
df_all.loc[mask_lugar2, 'v2a1'] = 100000.0
df_all.loc[mask_lugar3, 'v2a1'] = 100000.0
df_all.loc[mask_lugar4, 'v2a1'] = 80000.0
df_all.loc[mask_lugar5, 'v2a1'] = 65000.0
df_all.loc[mask_lugar6, 'v2a1'] = 70000.0

* vou imputar com -1 os NAs de 'rez_esc' por nao conseguir calcular a partir dos dados q tenho

In [ ]:
df_all['rez_esc'].fillna(-1, inplace=True)

In [ ]:
df_all.info(max_cols=145)

## Análise de Dados

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
sns.set_palette('RdBu')

### classes do dataset


In [ ]:
target= sns.countplot(x='Target',data=df_all)

target.set_title('Contagem de Classes da variável Target')
plt.xticks(np.arange(4), ['Extreme Poverty', 'Moderate Poverty', 'Vulnerable Household', 'Non-Vulnerable Household'], rotation=90)
plt.show()

### sobre as pessoas do dataset

In [ ]:
sex= sns.countplot(x='male',data=df_all)
sex.set_title('Sexo')
plt.xticks(np.arange(2), ['Feminino', 'Masculino'])
plt.show()

In [ ]:
sex= sns.countplot(x='dis',data=df_all, order=[1,0])
sex.set_title('Pessoa com Deficiência')
plt.xticks(np.arange(2), [ 'Sim', 'Não'])
plt.show()

In [ ]:
age=sns.histplot(df_all, x="age", bins=30 )
age.set_title('Distribuição das idades no dataset')
plt.show()

In [ ]:
ph=sns.countplot(x='hogar_total', hue='Target',data=df_all)
ph.set_title('Contagem de pessoas em casa por classes')
labels= ['Extreme Poverty', 'Moderate Poverty', 'Vulnerable Household', 'Non-Vulnerable Household']

plt.legend(loc='upper left', labels=labels)
plt.show()


In [ ]:
ph=sns.countplot(x='hogar_adul', hue='Target',data=df_all)
ph.set_title('Contagem de pessoas adultas(+19) em casa por classes')
labels= ['Extreme Poverty', 'Moderate Poverty', 'Vulnerable Household', 'Non-Vulnerable Household']

plt.legend(loc='upper left', labels=labels)
plt.show()


In [ ]:
ph=sns.countplot(x='hogar_nin', hue='Target',data=df_all)
ph.set_title('Contagem de pessoas entre 0-19 anos em casa por classes')
labels= ['Extreme Poverty', 'Moderate Poverty', 'Vulnerable Household', 'Non-Vulnerable Household']

plt.legend(loc='upper left', labels=labels)
plt.show()


In [ ]:
ph=sns.countplot(x='hogar_mayor', hue='Target',data=df_all)
ph.set_title('Contagem de pessoas idosos(+65) em casa por classes')
labels= ['Extreme Poverty', 'Moderate Poverty', 'Vulnerable Household', 'Non-Vulnerable Household']

plt.legend(loc='upper left', labels=labels)
plt.show()

In [ ]:

hep=sns.histplot(df_all[df_all['Target'] == 1], x="escolari", bins=30 )
hep.set_title('Anos de Escolaridade quando é Extrema Pobreza')
plt.show()

hp=sns.histplot(df_all[df_all['Target'] == 2], x="escolari", bins=30 )
hp.set_title('Anos de Escolaridade quando é Pobreza Moderada')
plt.show()

hcv=sns.histplot(df_all[df_all['Target'] == 3], x="escolari", bins=30 )
hcv.set_title('Anos de Escolaridade quando é Família Vulnerável')
plt.show()

hcnv=sns.histplot(df_all[df_all['Target'] == 4], x="escolari", bins=30 )
hcnv.set_title('Anos de Escolaridade quando é Família Não Vulnerável')
plt.show()

In [ ]:
tablet=sns.countplot(x="Target",hue="v18q",hue_order=[1,0], data=df_all)
tablet.set_title('Tem Tablet ou Não?')
plt.xticks(np.arange(4), ['Extreme Poverty', 'Moderate Poverty', 'Vulnerable Household', 'Non-Vulnerable Household'], rotation=90)

#legenda
plt.legend(title='Tablet', loc='upper left', labels=['Sim', 'Não'])
plt.show()

In [ ]:
cel=sns.countplot(x="Target",hue="mobilephone",hue_order=[1,0], data=df_all)
cel.set_title('Tem Celular ou Não?')
plt.xticks(np.arange(4), ['Extreme Poverty', 'Moderate Poverty', 'Vulnerable Household', 'Non-Vulnerable Household'], rotation=90)

#legenda
plt.legend(title='Celular', loc='upper left', labels=['Sim', 'Não'])
plt.show()

### sobre localização das casas

In [ ]:
area= sns.countplot(x='area1',data=df_all)
area.set_title('Zona da casa')
plt.xticks(np.arange(2), [ 'Zona Rural', 'Zona Urbana'])
plt.show()

In [ ]:
target= sns.countplot(x='lugar1', hue='Target',data=df_all[df_all['lugar1']==1])

target.set_title('Classes da variável Target por região')
plt.xticks(np.arange(1), ['Região Central'])
labels= ['Extreme Poverty', 'Moderate Poverty', 'Vulnerable Household', 'Non-Vulnerable Household']

plt.legend(loc='upper left', labels=labels)
plt.show()

#lugar2

target= sns.countplot(x='lugar2', hue='Target',data=df_all[df_all['lugar2']==1])

target.set_title('Classes da variável Target por região')
plt.xticks(np.arange(1), ['Região Chorotega'])
labels= ['Extreme Poverty', 'Moderate Poverty', 'Vulnerable Household', 'Non-Vulnerable Household']

plt.legend(loc='upper left', labels=labels)
plt.show()

# lugar3

target= sns.countplot(x='lugar3', hue='Target',data=df_all[df_all['lugar3']==1])

target.set_title('Classes da variável Target por região')
plt.xticks(np.arange(1), ['Região Pacífico Central'])
labels= ['Extreme Poverty', 'Moderate Poverty', 'Vulnerable Household', 'Non-Vulnerable Household']

plt.legend(loc='upper left', labels=labels)
plt.show()

#lugar4

target= sns.countplot(x='lugar4', hue='Target',data=df_all[df_all['lugar4']==1])

target.set_title('Classes da variável Target por região')
plt.xticks(np.arange(1), ['Região Brunca'])
labels= ['Extreme Poverty', 'Moderate Poverty', 'Vulnerable Household', 'Non-Vulnerable Household']

plt.legend(loc='upper left', labels=labels)
plt.show()

#lugar5

target= sns.countplot(x='lugar5', hue='Target',data=df_all[df_all['lugar5']==1])

target.set_title('Classes da variável Target por região')
plt.xticks(np.arange(1), ['Região Huetar Atlântica'])
labels= ['Extreme Poverty', 'Moderate Poverty', 'Vulnerable Household', 'Non-Vulnerable Household']

plt.legend(loc='upper left', labels=labels)
plt.show()

#lugar6

target= sns.countplot(x='lugar6', hue='Target',data=df_all[df_all['lugar6']==1])

target.set_title('Classes da variável Target por região')
plt.xticks(np.arange(1), ['Região Huetar Norte'])
labels= ['Extreme Poverty', 'Moderate Poverty', 'Vulnerable Household', 'Non-Vulnerable Household']

plt.legend(loc='upper left', labels=labels)
plt.show()

### sobre as casas

In [ ]:
ref=sns.countplot(x="Target",hue="refrig",hue_order=[1,0], data=df_all)
ref.set_title('Tem Geladeira ou Não?')
plt.xticks(np.arange(4), ['Extreme Poverty', 'Moderate Poverty', 'Vulnerable Household', 'Non-Vulnerable Household'], rotation=90)

#legenda
plt.legend(title='Geladeira', loc='upper left', labels=['Sim', 'Não'])
plt.show()

In [ ]:
ref=sns.countplot(x="Target",hue="v14a",hue_order=[1,0], data=df_all)
ref.set_title('Tem Sanitário na casa ou Não?')
plt.xticks(np.arange(4), ['Extreme Poverty', 'Moderate Poverty', 'Vulnerable Household', 'Non-Vulnerable Household'], rotation=90)

#legenda
plt.legend(title='Sanitário', loc='upper left', labels=['Sim', 'Não'])
plt.show()

In [ ]:
cama=sns.countplot(x="Target",hue="hacdor",hue_order=[1,0], data=df_all)
cama.set_title('Tem superlotação de camas ou Não?')
plt.xticks(np.arange(4), ['Extreme Poverty', 'Moderate Poverty', 'Vulnerable Household', 'Non-Vulnerable Household'], rotation=90)

#legenda
plt.legend(title='Superlotação', loc='upper left', labels=['Sim', 'Não'])
plt.show()

In [ ]:
quartos=sns.countplot(x="Target",hue="hacapo",hue_order=[1,0], data=df_all)
quartos.set_title('Tem superlotação de quartos ou Não?')
plt.xticks(np.arange(4), ['Extreme Poverty', 'Moderate Poverty', 'Vulnerable Household', 'Non-Vulnerable Household'], rotation=90)

#legenda
plt.legend(title='Superlotação', loc='upper left', labels=['Sim', 'Não'])
plt.show()

In [ ]:
comp=sns.countplot(x="Target",hue="computer",hue_order=[1,0], data=df_all)
comp.set_title('Tem computador ou Não?')
plt.xticks(np.arange(4), ['Extreme Poverty', 'Moderate Poverty', 'Vulnerable Household', 'Non-Vulnerable Household'], rotation=90)

#legenda
plt.legend(title='Computador', loc='upper left', labels=['Sim', 'Não'])
plt.show()

In [ ]:
tv=sns.countplot(x="Target",hue="television",hue_order=[1,0], data=df_all)
tv.set_title('Tem Televisão ou Não?')
plt.xticks(np.arange(4), ['Extreme Poverty', 'Moderate Poverty', 'Vulnerable Household', 'Non-Vulnerable Household'], rotation=90)

#legenda
plt.legend(title='Televisão', loc='upper left', labels=['Sim', 'Não'])
plt.show()

In [ ]:

hep=sns.histplot(df_all[df_all['Target'] == 1], x="overcrowding", bins=30 )
hep.set_title('Distribuição de pessoas por quarto quando é Extrema Pobreza')
plt.show()

hp=sns.histplot(df_all[df_all['Target'] == 2], x="overcrowding", bins=30 )
hp.set_title('Distribuição de pessoas por quarto quando é Pobreza Moderada')
plt.show()

hcv=sns.histplot(df_all[df_all['Target'] == 3], x="overcrowding", bins=30 )
hcv.set_title('Distribuição de pessoas por quarto quando é Família Vulnerável')
plt.show()

hcnv=sns.histplot(df_all[df_all['Target'] == 4], x="overcrowding", bins=30 )
hcnv.set_title('Distribuição de pessoas por quarto quando é Família Não Vulnerável')
plt.show()

### Preços de Alugueis

In [ ]:
hep=sns.histplot(df_all[df_all['Target'] == 1], x="v2a1", bins=30 )
hep.set_title('Preços de alugueis quando é Extrema Pobreza')
plt.show()

hp=sns.histplot(df_all[df_all['Target'] == 2], x="v2a1", bins=30 )
hp.set_title('Preços de alugueis quando é Pobreza Moderada')
plt.show()

hcv=sns.histplot(df_all[df_all['Target'] == 3], x="v2a1", bins=30 )
hcv.set_title('Preços de alugueis quando é Família Vulnerável')
plt.show()

hcnv=sns.histplot(df_all[df_all['Target'] == 4], x="v2a1", bins=30 )
hcnv.set_title('Preços de alugueis quando é Família Não Vulnerável')
plt.show()

## Separando entre treino e teste

In [ ]:
train, test = df_all[~df_all['Target'].isnull()], df_all[df_all['Target'].isnull()]

train.shape, test.shape

## Tratamento de classes desbalanceadas

In [ ]:
# Importando a biblioteca
import imblearn

In [ ]:
train['Target'].value_counts()

In [ ]:
train['Target'].value_counts(normalize=True)

In [ ]:
feats = [c for c in train.columns if c not in ['Id', 'idhogar', 'Target']]

In [ ]:
# Separando os dados de entrada e o target da base de treino
X, y = train[feats], train[['Target']]

In [ ]:
from imblearn.over_sampling import SMOTE

# Fazendo o SMOTE
sm = SMOTE()
X_sm, y_sm = sm.fit_resample(X,y)

# Verificando os dados
y_sm['Target'].value_counts()

In [ ]:
# Juntando os dados
train = pd.concat([X_sm, y_sm], axis=1)
train.shape, train['Target'].value_counts(normalize=True)

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Importando a métrica
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42, oob_score=True,
                           min_samples_leaf=5, min_samples_split=20, max_depth=10)

In [ ]:
# Separando as colunas para treinamento
feats2 = [c for c in df_all.columns if c not in ['Id', 'idhogar', 'Target']]

In [ ]:
# Treinando o modelo
rf.fit(train[feats], train['Target'])

In [ ]:
# Prever o Target do Dataframe de teste
test['Target'] = rf.predict(test[feats2]).astype(int)

In [ ]:
# Verificando as previsões
test['Target'].value_counts(normalize=True)

In [ ]:
# Criando o arquivo para submissão
test[['Id', 'Target']].to_csv('submission.csv', index=False) 

## GBM

In [ ]:
# Trabalhando com GBM
from sklearn.ensemble import GradientBoostingClassifier

gbm = GradientBoostingClassifier(n_estimators=200, learning_rate=1.0, max_depth=4, random_state=42, min_samples_split=2, min_samples_leaf=1, subsample=1)
gbm.fit(train[feats], train['Target'])

In [ ]:

# Prever o Target do Dataframe de teste
test['Target'] = gbm.predict(test[feats2]).astype(int)

In [ ]:
# Verificando as previsões
test['Target'].value_counts(normalize=True)

## XGboost

In [ ]:
# Trabalhando com XGBoost
from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators=200, learning_rate=0.09, random_state=42)
xgb.fit(train[feats], train['Target'])

In [ ]:
# Prever o Target do Dataframe de teste
test['Target'] = xgb.predict(test[feats2]).astype(int)

In [ ]:
# Verificando as previsões
test['Target'].value_counts(normalize=True)

## Adaboost

In [ ]:
# Trabalhando com AdaBoost
from sklearn.ensemble import AdaBoostClassifier

abc = AdaBoostClassifier(n_estimators=200, learning_rate=1.0, random_state=42)
abc.fit(train[feats], train['Target'])

In [ ]:
# Prever o Target do Dataframe de teste
test['Target'] = abc.predict(test[feats2]).astype(int)

In [ ]:
# Verificando as previsões
test['Target'].value_counts(normalize=True)

### O modelo random Forest teve maior score, foi o melhor modelo!!!